# Dash Interface

## Importing modules 

In [161]:
from os.path import dirname, join, isfile
from itertools import chain
import pandas as pd
import numpy as np
import scipy.signal
import scipy.io as sio
import matplotlib.pyplot as plt
from jupyter_dash import JupyterDash  # pip install dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Output, Input
import pandas as pd  # pip install pandas
import plotly.express as px
import math
from dash import no_update
from dash_canvas import DashCanvas

## Function to get dataframe

In [162]:
def get_dataframe(mat_fpath):
    ''' Obtain Pandas dataframe from mat file
    :param mat_fpath: File path to the mat file from which pandas DF should be created
    '''
    assert isfile(mat_fpath), "Invalid file path!"
    mat_contents = sio.loadmat(mat_fpath)
    viz_data = mat_contents["ccFddData"]
    viz_data = mat_contents['ccFddData']
    n = 5;
    iFDD_names = ["Sys2_Impact_Location_P{}".format(str(i+1)) for i in range(n)]
    dFDD_pnames = ["Sys2_Dmg_P_{}".format(str(i+1)) for i in range(n)]
    dFDD_cpnames = ["Sys2_Dmg_P_{}_c".format(str(i+1)) for i in range(n)]
    dFDD_names = list(chain.from_iterable(zip(dFDD_pnames, dFDD_cpnames)))
    col_names = ['Time', 'Sys2_Total_Dmg', 'Sys2_SimFDD_xe'] + iFDD_names + dFDD_names
    df = pd.DataFrame(viz_data, columns = col_names)
    return df             

In [163]:
data = get_dataframe("newData_v6.mat")

In [164]:
t = data['Time'].to_numpy()
trueHS = data['Sys2_Total_Dmg'].to_numpy()
sim_fdd_xe = data['Sys2_SimFDD_xe'].to_numpy()
impact_fdd = np.vstack(([data['Sys2_Impact_Location_P{}'.format(ii+1)].to_numpy() for ii in range(5)])).T
dmg_fdd = [np.vstack((data['Sys2_Dmg_P_{}'.format(ii+1)].to_numpy(), \
                     data['Sys2_Dmg_P_{}_c'.format(ii+1)].to_numpy())).T for ii in range(5)]

## Calculating time of impact and the probability around it

In [165]:
x = (int(np.argmax(impact_fdd))) / 5 #Getting the index for the peak registered for the impact FDD
timeofImpact = int(x)
timeofImpact

prob = (impact_fdd[timeofImpact]) #list with impact probabilities for all 5 segments
domeaff = [] #list with index of segments in which impact was registered
for i in range(5):
  if prob[i] > 0.2:
    domeaff.append(i)

#translating prabiblities into words
for i in domeaff:
  if prob[i] < 0.4:
    confidence = "probably not"
  elif prob[i] < 0.6:
    confidence = "chances about even"
  elif prob[i] < 0.8:
    confidence = "probablt"
  else:
    confidence = "almost certainly" 



## Creating lists to store information

In [166]:
height = np.zeros(5) #list that will store the average values
stdv = np.zeros(5) #list that will store standard deviation values
min_ = np.zeros(5) #list that will contain the min values for the data 
max_ = np.zeros(5) #list that will contain the max values for the data
st_error = np.zeros(5) #list that contains the standard error

## Function to calculate statiscs of data

In [167]:
def avg_std2(dmg_info, impact, num, sec1, sec2): #function that calculates the mean between to time intervals
  a = (dmg_info)
  b = a[(sec1 * 1000):(sec2 * 1000),0]
  min_[num] = np.min(b) #appends minimum value for each segment
  max_[num] = np.max(b) #appends maximum value for each segment
  height[num] = np.mean(b) #calculates mean of set of values
  stdv[num] = np.std(b) #calculates std dev. of set of values
  st_error = np.std(b, ddof=1) / np.sqrt(np.size(b)) #calculates the standard error for each segment
  num += 1;
  return num;

## Function to cap std on bar chart graph at 0 and 1

In [168]:
def lin(std, h): #function that limit standard deviations to between one and zero
# takes standard deviation and height of the bars and makes sure the height +- the standard
#deviation is within that range. Returns list with values to use
  fr = np.zeros(10)
  index = 0
  for i in std:
    if index < 5:
      if (h[index] - i) < 0:
        fr[index] = h[index]
      else:
        fr[index] = i
    else:
      if (h[index] + i > 1):
        fr[index] = 1 - h[index]
      else:
        fr[index] = i
    index = index + 1

  return (fr)

## Function to call other functions

In [169]:
def pregraph2(t1, t2):
#function that calls avg_std2 function, and adjusts the standard deviation
#to be within 0 and 1

  count = 0;
  counter = avg_std2(dmg_fdd[0], timeofImpact, count, t1, t2);
  counter = avg_std2(dmg_fdd[1], timeofImpact, counter, t1, t2);
  counter = avg_std2(dmg_fdd[2], timeofImpact, counter, t1, t2);
  counter = avg_std2(dmg_fdd[3], timeofImpact, counter, t1, t2);
  counter = avg_std2(dmg_fdd[4], timeofImpact, counter, t1, t2);
  aaa = np.concatenate((stdv, stdv))
  bbb = np.concatenate((height, height))
  s = lin(aaa, bbb)
  c = np.array_split(s, 2)
  return c

## Function to graph bar charts

In [170]:
def graph3(start_time, end_time): #function that graphs results, returns figure
  ccc = pregraph2(start_time, end_time)
  seg = ["segment 1", "segment 2", "segment 3", "segment 4", "segment 5"]
  fig2 = px.bar(x=seg, y=height, error_y=ccc[1], error_y_minus=ccc[0],
                title=f"Probability of damage in each segment from {start_time} seconds until {end_time} seconds",
                range_y = [0, 1], labels={"y": "P [Damage]", "x": ""})
            
  
  
  return fig2

## Function to graph FDD data

In [171]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
#statically graphs FDD data in a subplot
def FDD_graphs(st, ft):
  fig = make_subplots(rows=3, cols=2, 
                      subplot_titles=("Damage FDD: Segment 1", "Damage FDD: Segment 2", 
                      "Damage FDD: Segment 3", "Damage FDD: Segment 4", 
                      "Damage FDD: Segment 5"))


  fig.add_trace(
      go.Scatter(x = t[st*1000:ft*1000], y=dmg_fdd[0][st*1000:ft*1000,0], name="P[Damage] seg 1"),
      row = 1, col = 1
  )
  fig.add_trace(
      go.Scatter(x =t[st*1000:ft*1000], y=dmg_fdd[0][st*1000:ft*1000,1], name="P[No Damage] seg 1"),
      row = 1, col = 1
  )
  fig.add_trace(
      go.Scatter(x =t[st*1000:ft*1000], y=dmg_fdd[1][st*1000:ft*1000,0], name="P[Damage] seg 2"),
      row = 1, col = 2
  )
  fig.add_trace(
      go.Scatter(x = t[st*1000:ft*1000], y=dmg_fdd[1][st*1000:ft*1000,1], name="P[No Damage] seg 2"),
      row = 1, col = 2
  )
  fig.add_trace(
      go.Scatter(x = t[st*1000:ft*1000], y=dmg_fdd[2][st*1000:ft*1000,0], name="P[Damage] seg 3"),
      row = 2, col = 1
  )
  fig.add_trace(
      go.Scatter(x = t[st*1000:ft*1000], y=dmg_fdd[2][st*1000:ft*1000,1], name="P[No Damage] seg 3"),
      row = 2, col = 1
  )
  fig.add_trace(
      go.Scatter(x = t[st*1000:ft*1000], y=dmg_fdd[3][st*1000:ft*1000,0], name="P[Damage] seg 4"),
      row = 2, col = 2
  )
  fig.add_trace(
      go.Scatter(x = t[st*1000:ft*1000], y=dmg_fdd[3][st*1000:ft*1000,1], name="P[No Damage] seg 4"),
      row = 2, col = 2
   )
  fig.add_trace(
      go.Scatter(x = t[st*1000:ft*1000], y=dmg_fdd[4][st*1000:ft*1000,0], name="P[Damage] seg 5"),
      row = 3, col = 1
  )
  fig.add_trace(
      go.Scatter(x = t[st*1000:ft*1000], y=dmg_fdd[4][st*1000:ft*1000,1], name="P[No Damage] seg 5"),
      row = 3, col =1
  )


  #fig.update_layout(height=1000, width=900, title_text="FDD data for damage in each segment")
  return fig

## Creating explanation figure for bar and line chart
Gets pngs from github repository in which pictures are uploaded

In [172]:
# Create figure
fig34 = go.Figure()

# Constants
img_width = 2339
img_height = 1654
scale_factor = 0.5

# Add invisible scatter trace.
# This trace is added to help the autoresize logic work.
fig34.add_trace(
    go.Scatter(
        x=[0, img_width * scale_factor],
        y=[0, img_height * scale_factor],
        mode="markers",
        marker_opacity=0
    )
)

# Configure axes
fig34.update_xaxes(
    visible=False,
    range=[0, img_width * scale_factor]
)

fig34.update_yaxes(
    visible=False,
    range=[0, img_height * scale_factor],
    # the scaleanchor attribute ensures that the aspect ratio stays constant
    scaleanchor="x"
)

# Add image
fig34.add_layout_image(
    dict(
        x=0,
        sizex=img_width * scale_factor,
        y=img_height * scale_factor,
        sizey=img_height * scale_factor,
        xref="x",
        yref="y",
        opacity=1.0,
        layer="below",
        sizing="stretch",
        source="https://raw.githubusercontent.com/BernardoAlvarenga/picture/main/test%20(1)-1.png")
)

# Configure other layout
fig34.update_layout(
    width=img_width * scale_factor,
    height=img_height * scale_factor,
    margin={"l": 0, "r": 0, "t": 0, "b": 0},
)


In [173]:
# Create figure
fig35 = go.Figure()

# Constants
img_width = 2339
img_height = 1654
scale_factor = 0.5

# Add invisible scatter trace.
# This trace is added to help the autoresize logic work.
fig35.add_trace(
    go.Scatter(
        x=[0, img_width * scale_factor],
        y=[0, img_height * scale_factor],
        mode="markers",
        marker_opacity=0
    )
)

# Configure axes
fig35.update_xaxes(
    visible=False,
    range=[0, img_width * scale_factor]
)

fig35.update_yaxes(
    visible=False,
    range=[0, img_height * scale_factor],
    # the scaleanchor attribute ensures that the aspect ratio stays constant
    scaleanchor="x"
)

# Add image
fig35.add_layout_image(
    dict(
        x=0,
        sizex=img_width * scale_factor,
        y=img_height * scale_factor,
        sizey=img_height * scale_factor,
        xref="x",
        yref="y",
        opacity=1.0,
        layer="below",
        sizing="stretch",
        source="https://raw.githubusercontent.com/BernardoAlvarenga/picture/main/Line%20Graphs%20Explanation-1.png")
)

# Configure other layout
fig35.update_layout(
    width=img_width * scale_factor,
    height=img_height * scale_factor,
    margin={"l": 0, "r": 0, "t": 0, "b": 0},
)

# Disable the autosize on double click because it adds unwanted margins around the image
# More detail: https://plotly.com/python/configuration-options/


## Creating Line Graphs (real time + static)

In [174]:
from itertools import count
#Initializations of arrays and values for calculations
x = (int(np.argmax(impact_fdd))) / 5 #Getting the index for the peak registered for the impact FDD
timeofImpact = int(x)
default_interval = 3.0*1000 #Default interval of time for averaging and displaying on line graph
sec_ai = 60 #seconds after impact calculations will be done


cnt = 0

def pregraph4(it, ft):
  global average
  average = [[], [], [], [], []]
  global stddev
  stddev = [[], [], [], [], []]
  cnt = 0
  global low_1
  global low_2
  global low_3
  global low_4
  global low_5
  global high_1
  global high_2
  global high_3
  global high_4
  global high_5
  low_1 = []
  low_2 = []
  low_3 = []
  low_4 = []
  low_5 = []
  high_1 = []
  high_2 = []
  high_3 = []
  high_4 = []
  high_5 = []
  

  #Calculations of averages of windows over time, values stored in average and stddev lists
  def avg_std(dmg_info, impact, cnt, st, ft): #function that calculates the mean and standard deviation
  # for a certain time interval, can be adjusted
    a = (dmg_info[st*1000:])
    b = a[:((ft-st) * 1000),0]
    
    for i in range(0, len(b), 1000):
        
        if (i - default_interval) < 0:
            #print(np.mean(b[impact:i]))
            if (i == 0):
              average[cnt].append(np.mean(b[0]))
              stddev[cnt].append(np.std(b[0]))
            else:
              average[cnt].append(np.mean(b[0:i]))
              stddev[cnt].append(np.std(b[0:i]))
        else:
            average[cnt].append(np.mean(b[(i-(int(3.0*1000))):i]))
            stddev[cnt].append(np.std(b[(i-(int(3.0*1000))):i]))

  #Function Calls and List Initializations
  avg_std(dmg_fdd[0], timeofImpact, 0, it, ft);
  avg_std(dmg_fdd[1], timeofImpact, 1, it, ft);
  avg_std(dmg_fdd[2], timeofImpact, 2, it, ft);
  avg_std(dmg_fdd[3], timeofImpact, 3, it, ft);
  avg_std(dmg_fdd[4], timeofImpact, 4, it, ft);


  #Appending lower and upper bounds to repsective lists, capping at 0 and 1
  for i in range(len(average[0])):
    low_1.append(average[0][i] - stddev[0][i])
    low_2.append(average[1][i] - stddev[1][i])
    low_3.append(average[2][i] - stddev[2][i])
    low_4.append(average[3][i] - stddev[3][i])
    low_5.append(average[4][i] - stddev[4][i])
    high_1.append(average[0][i] + stddev[0][i])
    high_2.append(average[1][i] + stddev[1][i])
    high_3.append(average[2][i] + stddev[2][i])
    high_4.append(average[3][i] + stddev[3][i])
    high_5.append(average[4][i] + stddev[4][i])
    low_list = [low_1, low_2, low_3, low_4, low_5]
    high_list = [high_1, high_2, high_3, high_4, high_5]
    for k in low_list:
      if k[i] < 0:
        k[i] = 0
    for j in high_list:
      if j[i] > 1:
        j[i] = 1

  #Low lists reversed
  low_1 = low_1[::-1]
  low_2 = low_2[::-1]
  low_3 = low_3[::-1]
  low_4 = low_4[::-1]
  low_5 = low_5[::-1]

  #Iteration for x axis list
  index = count(start=it)
  global xvals
  xvals = [0]*len(average[0]) 
  for i in range(len(average[0])):
    xvals[i] = next(index)

In [175]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
def avg_linegraphs(it, ft):
    pregraph4(it, ft)
    fig = go.Figure()   
    fig.add_trace(go.Scatter(
        x=xvals+xvals[::-1],
        y=high_1+low_1,
        fill='toself',
        fillcolor='rgba(34, 139, 34, 0.2)',
        line_color='rgba(255,255,255,0)',
        showlegend=False,
        name='Segment 1',
    ))
    fig.add_trace(go.Scatter(
        x=xvals+xvals[::-1],
        y=high_2+low_2,
        fill='toself',
        fillcolor='rgba(0,176,246,0.2)',
        line_color='rgba(255,255,255,0)',
        name='Segment 2',
        showlegend=False,
    ))
    fig.add_trace(go.Scatter(
        x=xvals+xvals[::-1],
        y=high_3+low_3,
        fill='toself',
        fillcolor='rgba(255,211,67,0.2)',
        line_color='rgba(255,255,255,0)',
        showlegend=False,
        name='Segment 3',
    ))
    fig.add_trace(go.Scatter(
        x=xvals+xvals[::-1],
        y=high_4+low_4,
        fill='toself',
        fillcolor='rgba(214, 39, 40, 0.2)',
        line_color='rgba(255,255,255,0)',
        showlegend=False,
        name='Segment 4',
    ))
    fig.add_trace(go.Scatter(
        x=xvals+xvals[::-1],
        y=high_5+low_5,
        fill='toself',
        fillcolor='rgba(255, 127, 14, 0.2)',
        line_color='rgba(255,255,255,0)',
        showlegend=False,
        name='Segment 5',
    ))
    fig.add_trace(go.Scatter(
        x=xvals, y=average[0],
        line_color='rgb(34, 139, 34)',
        name='Segment 1',
    ))
    fig.add_trace(go.Scatter(
        x=xvals, y=average[1],
        line_color='rgb(0,176,246)',
        name='Segment 2',
    ))
    fig.add_trace(go.Scatter(
        x=xvals, y=average[2],
        line_color='rgb(255,211,67)',
        name='Segment 3',
    ))
    fig.add_trace(go.Scatter(
        x=xvals, y=average[3],
        line_color='rgb(214, 39, 40)',
        name='Segment 4',
    ))
    fig.add_trace(go.Scatter(
        x=xvals, y=average[4],
        line_color='rgb(255, 127, 14)',
        name='Segment 5',
    ))
    

    fig.update_traces(mode='lines')
    return fig

## Creating real time impact FDD graphs
For this current iteration impact fdd data had to be simplified to accomodate for animation. Only 1 point for each second is shown, even though there are 1000 data points per second

In [176]:
def graphiFDD(st, ft):
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go
    fig43 = make_subplots(
        rows=5, cols=1, subplot_titles=('Segment 1', 'Segment 2', 'Segment 3', 'Segment 4', 'Segment 5'),
        horizontal_spacing=0.051, shared_xaxes = True, shared_yaxes = True
    )

    fig43.add_trace(go.Scatter(x=np.arange(0,120, 0.001), y = impact_fdd[1000*st:1000*ft, 0], mode="lines", name="P[Damage] seg1"), row=1, col=1) #this is the trace of index 0

    fig43.add_trace(go.Scatter(x=np.arange(0,120, 0.001),y = impact_fdd[1000*st:1000*ft, 1], mode="lines", name="P[Damage] seg2"), row=2, col=1)  #trace of index 1
    fig43.add_trace(go.Scatter(x=np.arange(0,120, 0.001),y = impact_fdd[1000*st:1000*ft, 2], mode="lines", name="P[Damage] seg3"), row=3, col=1) #this is the trace of index 2

    fig43.add_trace(go.Scatter(x=np.arange(0,120, 0.001),y = impact_fdd[1000*st:1000*ft, 3], mode="lines", name="P[Damage] seg4"), row=4, col=1)  #trace of index 3

    fig43.add_trace(go.Scatter(x=np.arange(0,120, 0.001),y = impact_fdd[1000*st:1000*ft, 4], mode="lines", name="P[Damage] seg5"), row=5, col=1) #this is the trace of index 4
    fig43.update_layout(yaxis2_range=[0,1], yaxis_range=[0,1],
                    yaxis3_range=[0,1], yaxis5_range=[0,1],yaxis4_range=[0,1],
                    title="Impact probabilities for each segment", xaxis5_title="Time",
                    yaxis3_title="P[Damage]")
    return fig43

 

## Dash interface 2.0

In [178]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
t_update = 8


app.layout = html.Div([
  #Interval strcutures that updates every 10 seconds with more data
  #to change time to take for app to update change the variable ttu (time to update above)
  dcc.Interval(
    id="ult_timetrack",
    interval=t_update*1000,
    n_intervals=0),
  #high level tabs. Currently divided into after impact info and real time status
  dcc.Tabs(id='tabs-main', value='tab-dmg', children=[
        ##Structures inside After Impact Information Tab
        dcc.Tab(label='After impact Info', value='tab-main' ,children=        
                html.Div([
                #Dropdown to select graph to display
                dcc.Dropdown(
                    id='my_dropdown',
                    options=[
                              {"label": "See damage history data", "value": "linechart"},
                              {"label": "Show estimated damage", "value": "barchart"},   
                              {"label": "See FDD data", "value": "FDDdata"}, 
                              ],
                    optionHeight=35,                    #height/space between dropdown options
                    value='barchart',                    #dropdown value selected automatically when page loads
                    disabled=False,                     #disable dropdown value selection
                    multi=False,                        #allow multiple dropdown values to be selected
                    searchable=True,                    #allow user-searching of dropdown values
                    search_value='',                    #remembers the value searched in dropdown
                    placeholder='Please select...',     #gray, default text shown when no option is selected
                    clearable=True,                     #allow user to removes the selected value
                    style={'width':"100%", 'display': 'inline-block'},             #use dictionary to define CSS styles of your dropdown         
                  ),
                  #graph displayed below dropbar, changes according to user's choice
                  dcc.Graph(
                      id="graph_selected",
                      figure={}
                  ),
                  #Range slider, provides user with ability to choose time interval to go over
                  dcc.RangeSlider(
                      id="time_interval",
                      min=0,
                      step=1,
                      value=[0, 1]  
                  ),
                  ##Tabs inside After impact Information showing explanation for graphs shown. Some are still missing info
                  dcc.Tabs(id='tabs-example', value='tab-1', children=[
                      dcc.Tab(label='How estimated damage was calculated', value='tab-1' ,children=
                              dcc.Graph(
                                  id="exp1",
                                  figure=fig34
                                  )
                              ),
                      dcc.Tab(label='How damage FDD data was calculated', value='tab-2'),
                      dcc.Tab(label="How real time feedback was calculated", value="tab-3", children=
                              dcc.Graph(
                                  id="exp2",
                                  figure = fig35
                              )
                              ),
                      dcc.Tab(label="How Impact FDD data was calculated", value="tab4")
                      ]),
                  html.Div(id='dd-output-container')])
                              ),
        ##tab that shows real time status
        dcc.Tab(label="Real time status", value="tab-dmg", children=
                html.Div([
                          #Graph to show impact FDD
                          dcc.Graph(
                              id="rtimpactdata"
                              
                          ),
                          #Graph to show line graph heuristic of damageFDD
                          dcc.Graph(
                              id="rtdamagedata"
                              
                          )
                ]))
  
         ]),
    html.Div(id='outer-tab')
  ])
## Callback is an element of a dash app that updates the app each time any of the inputs change. Once that is the case
## the function below is runned and the values returned are assigned to the outputs
#outputs and inputs consist of strcuture id and component that it is refering to
@app.callback(
    [Output("graph_selected", "figure"), 
    Output("rtimpactdata", "figure"), Output("rtdamagedata", "figure"), Output("time_interval", "max"),
    Output("time_interval", "marks")],
    [Input('my_dropdown', 'value'), Input("time_interval", "value"), 
    Input("tabs-example", "value"), Input("ult_timetrack", "n_intervals")]
    
    )
  ## Function to update values
def update_output(value, time_int, tabss, n):
    if value == "barchart":
      fig = graph3(time_int[0], time_int[1])
    elif value == "FDDdata":
      fig = FDD_graphs(0, t_update*n)
    elif value == "linechart":
      fig = avg_linegraphs(time_int[0], time_int[1])
    fig1 = avg_linegraphs(0, t_update*n)
    fig2 = graphiFDD(0,t_update*n)
    m = t_update*n
    marks = {0: str(-t_update*n)}
    return  fig, fig2, fig1, m, marks


app.run_server(debug=False, port=8040)


127.0.0.1 - - [28/Jul/2021 09:33:57] "GET /_shutdown_ab67edb4-2bfc-47a5-b58a-72609532b4d5 HTTP/1.1" 200 -
 * Running on http://127.0.0.1:8040/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Jul/2021 09:33:58] "GET /_alive_ab67edb4-2bfc-47a5-b58a-72609532b4d5 HTTP/1.1" 200 -


Dash app running on http://127.0.0.1:8040/


127.0.0.1 - - [28/Jul/2021 09:34:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2021 09:34:12] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2021 09:34:12] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2021 09:34:12] "GET /_dash-component-suites/dash_core_components/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2021 09:34:13] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [28/Jul/2021 09:34:13] "GET /_dash-component-suites/dash_core_components/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2021 09:34:21] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [28/Jul/2021 09:34:29] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [28/Jul/2021 09:34:37] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [28/Jul/2021 09:34:46] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [28/Jul/2021 09:34:53] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [28/Jul/2021 09:34:53] "GET /_dash-comp